In [ ]:
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()


Saving merged_dataset.csv to merged_dataset.csv


Saving test.csv to test.csv


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
label_map = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [ ]:
df = pd.read_csv('merged_dataset.csv')

In [ ]:
texts = df['conversation'].astype(str).tolist()
labels = df['class'].astype(str).tolist()

In [ ]:
# 1) 원하는 순서대로 매핑 딕셔너리 정의
label_map = {
    '협박 대화':            0,
    '갈취 대화':            1,
    '직장 내 괴롭힘 대화':  2,
    '기타 괴롭힘 대화':     3,
    '일반대화':            4
}

# 2) 원본 레이블 리스트(labels)를 숫자로 변환
y = [ label_map[label] for label in labels ]

# 3) 클래스 개수 설정
num_classes = len(label_map)


In [ ]:
MAX_LEN = 512

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

VOCAB_SIZE = 20000
tokenizer_ks = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer_ks.fit_on_texts(X_train)

train_seq = tokenizer_ks.texts_to_sequences(X_train)
test_seq  = tokenizer_ks.texts_to_sequences(X_test)
train_seq = pad_sequences(train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
test_seq  = pad_sequences(test_seq,  maxlen=MAX_LEN, padding="post", truncating="post")

In [ ]:

TOKENIZER_NAME = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(TOKENIZER_NAME)

def encode_bert(texts):
    enc = tokenizer(
        texts,
        max_length=MAX_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return enc["input_ids"], enc["attention_mask"]

train_ids, train_mask = encode_bert(X_train)
test_ids,  test_mask  = encode_bert(X_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
# 4. KoBERT 서브클래싱 모델 정의
class KoBertClassifier(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(TOKENIZER_NAME)
        self.dense1 = tf.keras.layers.Dense(128, activation="relu")
        self.drop   = tf.keras.layers.Dropout(0.3)
        self.out    = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        # inputs: dict with "input_ids" and "attention_mask"
        bert_out = self.bert(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            training=training
        )
        cls_tok = bert_out.last_hidden_state[:,0,:]      # [CLS] 토큰 임베딩
        x = self.dense1(cls_tok)
        x = self.drop(x, training=training)
        return self.out(x)

In [ ]:
model_bert = KoBertClassifier(num_classes)
model_bert.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
history_bert = model_bert.fit(
    {"input_ids": train_ids, "attention_mask": train_mask},
    np.array(y_train),
    validation_split=0.1,
    epochs=10,
    batch_size=4,
    verbose=1
)

Epoch 1/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 77s 35ms/step - accuracy: 0.4405 - loss: 1.3976 - val_accuracy: 0.5385 - val_loss: 1.0419
Epoch 2/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5636 - loss: 1.1060 - val_accuracy: 0.5990 - val_loss: 0.9332
Epoch 3/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5944 - loss: 1.0178 - val_accuracy: 0.6252 - val_loss: 0.8871
Epoch 4/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5820 - loss: 1.0045 - val_accuracy: 0.6465 - val_loss: 0.8556
Epoch 5/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.6180 - loss: 0.9500 - val_accuracy: 0.6465 - val_loss: 0.8381
Epoch 6/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.6095 - loss: 0.9454 - val_accuracy: 0.6759 - val_loss: 0.8098
Epoch 7/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.6323 - loss: 0.9003 - val_accuracy: 0.6710 - val_loss: 0.7999
Epoch 8/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.6341 -

In [ ]:
# 6. BiLSTM 모델 정의 & 학습
from tf_keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras import Input, Model

seq_input = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32)  # 여기만 변경

x = Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN)(seq_input)
x = Bidirectional(LSTM(128, return_sequences=False))(x)
x = Dropout(0.3)(x)
out = Dense(num_classes, activation="softmax")(x)

earlystop = EarlyStopping(
    monitor="val_loss",
    patience=1,
    restore_best_weights=True
)

model_lstm = tf.keras.Model(seq_input, out)
model_lstm.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


history_lstm = model_lstm.fit(
    train_seq,
    np.array(y_train),
    validation_split=0.1,
    epochs=10,       # 충분히 크게
    batch_size=16,
    callbacks=[earlystop],
    verbose=1
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.5737 - loss: 1.0417 - val_accuracy: 0.7774 - val_loss: 0.5208
Epoch 2/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.8458 - loss: 0.4012 - val_accuracy: 0.8511 - val_loss: 0.3809
Epoch 3/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.9491 - loss: 0.1674 - val_accuracy: 0.8560 - val_loss: 0.3928


In [ ]:
# 4) 모델별 예측 확률
# Assume you want to predict on the test set
probs_bert = model_bert.predict(
    {"input_ids": test_ids, "attention_mask": test_mask}, # Use test_ids and test_mask
    batch_size=16, verbose=1
)
probs_lstm = model_lstm.predict(test_seq, batch_size=32, verbose=1) # Use test_seq for LSTM

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [ ]:
from transformers import AutoTokenizer, TFRobertaForSequenceClassification, RobertaConfig, create_optimizer
import math
import numpy as np

# 1) AutoTokenizer 사용
roberta_name = "klue/roberta-base"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_name)

def encode_roberta(texts, max_len=MAX_LEN):
    enc = roberta_tokenizer(
        texts,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return enc["input_ids"], enc["attention_mask"]

train_ids_r, train_mask_r = encode_roberta(X_train)
test_ids_r,  test_mask_r  = encode_roberta(X_test)

# 2) 모델 정의 (나머지 동일)
config_r = RobertaConfig.from_pretrained(
    roberta_name,
    num_labels=num_classes,
    hidden_dropout_prob=0.3,
    attention_probs_dropout_prob=0.3
)
model_roberta = TFRobertaForSequenceClassification.from_pretrained(
    roberta_name,
    config=config_r
)

# 3) 옵티마이저/스케줄러 설정
batch_size_r   = 4
epochs_r       = 4
steps_r        = math.ceil(len(train_ids_r) / batch_size_r) * epochs_r
warmup_r       = int(0.1 * steps_r)
optim_r, _     = create_optimizer(
    init_lr=2e-5,
    num_train_steps=steps_r,
    num_warmup_steps=warmup_r,
    weight_decay_rate=0.01
)

model_roberta.compile(
    optimizer=optim_r,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 4) 학습 & 예측, 앙상블까지는 기존과 동일하게 진행하시면 됩니다.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
# 3) KLUE-RoBERTa 학습

history_roberta = model_roberta.fit(
    {"input_ids": train_ids_r, "attention_mask": train_mask_r},
    np.array(y_train),
    validation_split=0.1,
    epochs=3,
    batch_size=batch_size_r,
    callbacks=[],
    verbose=1
)

Epoch 1/3
1375/1375 [==============================] - 138s 85ms/step - loss: 2.0255 - accuracy: 0.4623 - val_loss: 0.8994 - val_accuracy: 0.6367
Epoch 2/3
1375/1375 [==============================] - 114s 83ms/step - loss: 0.8165 - accuracy: 0.7095 - val_loss: 0.4494 - val_accuracy: 0.9034
Epoch 3/3
1375/1375 [==============================] - 114s 83ms/step - loss: 0.6266 - accuracy: 0.8219 - val_loss: 0.6043 - val_accuracy: 0.7660


In [ ]:
# 4) KLUE-RoBERTa 예측
probs_roberta = model_roberta.predict(
    {"input_ids": test_ids_r, "attention_mask": test_mask_r},
    batch_size=16,
    verbose=1
)


32/32 [==============================] - 3s 81ms/step


In [ ]:
# 5) 3-way 앙상블 (KoBERT + BiLSTM + RoBERTa)
w_bert, w_lstm, w_rob = 0.2, 0.5, 0.3
probs_ensemble = (
      w_bert * probs_bert
    + w_lstm * probs_lstm
    + w_rob  * probs_roberta.logits # Access the logits from probs_roberta
)
pred_ids = np.argmax(probs_ensemble, axis=1)

# *** Insert the following code ***

# Adjust the length of y_test to match the prediction size
y_test = y_test[:len(pred_ids)]
y_test_array = np.array(y_test)

# Now continue with your existing code

# 1) argmax로 예측 클래스, max로 확신도(confidence) 추출
pred_ids   = np.argmax(probs_ensemble, axis=1)   # shape=(num_samples,)
pred_confs = np.max(probs_ensemble, axis=1)      # shape=(num_samples,)

# 2) 원하는 threshold 값 설정
threshold = 0.7

# 3) threshold 이상인 샘플만 pred_ids, 아니면 -1로 marking
#    -1은 “불확실” 표시용. 다른 값(예: num_classes)으로 바꿔도 무방.
final_preds = np.where(pred_confs >= threshold, pred_ids, -1)

# 4) (선택) 불확실 샘플 개수 확인
num_total     = final_preds.shape[0]
num_uncertain = np.sum(final_preds == -1)
print(f"[Threshold={threshold}] 전체: {num_total}, 불확실(-1): {num_uncertain}")

# 5) (선택) 불확실 샘플 제외한 평가
from sklearn.metrics import classification_report

mask = final_preds != -1

print(classification_report(
    y_test_array[mask], # 인덱싱에 y_test_array를 사용합니다.
    final_preds[mask],
    target_names=list(label_map.keys()),
    digits=4
))

[Threshold=0.7] 전체: 500, 불확실(-1): 256
              precision    recall  f1-score   support

       협박 대화     0.2000    0.1176    0.1481        34
       갈취 대화     0.2308    0.0909    0.1304        33
 직장 내 괴롭힘 대화     0.1429    0.3235    0.1982        34
   기타 괴롭힘 대화     0.1789    0.5789    0.2733        38
        일반대화     0.4545    0.0476    0.0862       105

    accuracy                         0.1844       244
   macro avg     0.2414    0.2317    0.1673       244
weighted avg     0.3024    0.1844    0.1456       244



In [ ]:
# 5) 3-way 앙상블 (KoBERT + BiLSTM + RoBERTa)
w_bert, w_lstm, w_rob = 0.2, 0.5, 0.3
probs_ensemble = (
      w_bert * probs_bert
    + w_lstm * probs_lstm
    + w_rob  * probs_roberta.logits # Access the logits from probs_roberta
)
pred_ids = np.argmax(probs_ensemble, axis=1)

In [ ]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 5-1) X_test / y_test 에 대한 F1 계산
f1 = f1_score(y_test, pred_ids, average="macro")
print(f"Ensemble Macro-F1 on held-out test: {f1:.4f}")

# 5-2) 분류 리포트
print("\n=== Classification Report ===")
print(classification_report(
    y_test,
    pred_ids,
    target_names=list(label_map.keys()),  # ["협박","갈취","직장 내 괴롭힘","기타 괴롭힘","일반대화"]
    digits=4
))

# 5-3) 혼동 행렬
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, pred_ids))


ValueError: Found input variables with inconsistent numbers of samples: [1528, 500]

In [ ]:
# Kaggle test.csv 예측 & Submission 생성
TEST_PATH = "/content/test.csv"
test_df   = pd.read_csv(TEST_PATH)
texts     = test_df["text"].astype(str).tolist()

# KoBERT 예측
test_ids, test_mask = encode_bert(texts)
probs_bert = model_bert.predict({"input_ids": test_ids, "attention_mask": test_mask}, batch_size=8) # Reduced batch size to 8

# BiLSTM 예측
raw_seq    = tokenizer_ks.texts_to_sequences(texts)
test_seq   = pad_sequences(raw_seq, maxlen=MAX_LEN, padding="post", truncating="post")
probs_lstm = model_lstm.predict(test_seq, batch_size=16) # Reduced batch size to 16

# RoBERTa 예측
test_ids_r, test_mask_r = encode_roberta(texts)
# Reduced batch size to 4 and added try-except block for handling potential memory issues.
# **Change: Removed batch_size from model_roberta call and added it to predict**
try:
  probs_roberta = model_roberta.predict({"input_ids": test_ids_r, "attention_mask": test_mask_r}, batch_size=4)
  probs_roberta = tf.nn.softmax(probs_roberta.logits, axis=-1).numpy() # Access logits after prediction
except tf.errors.ResourceExhaustedError:
  print("GPU memory exhausted. Consider reducing the batch size or sequence length further.")
  # Implement a fallback strategy, e.g., break the data into smaller chunks and process them sequentially.

# 3-way 앙상블
w_bert, w_lstm, w_rob = 0.2, 0.7, 0.1
final_probs = w_bert*probs_bert + w_lstm*probs_lstm + w_rob*probs_roberta
pred_ids    = np.argmax(final_probs, axis=1)

# Submission 저장
submission = pd.DataFrame({"idx": test_df["idx"], "target": pred_ids})
submission.to_csv("/content/three_model_submission.csv", index=False)
print("✅ three_submission.csv 저장 완료")

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
125/125 [==============================] - 4s 31ms/step
✅ three_submission.csv 저장 완료


In [ ]:
import pandas as pd

# 1) 파일 경로 수정
file_path = "/content/three_model_submission.csv"  # Colab 환경에 맞게 조정

# 2) CSV 로드
df = pd.read_csv(file_path)

# 3) target 값별 개수 계산
dist = df["target"].value_counts().sort_index()

# 4) 결과 출력
print("=== Target Value Distribution ===")
print(dist)

# 5) (선택) DataFrame 형태로 보기
dist_df = dist.reset_index()
dist_df.columns = ["target", "count"]
print("\n=== As DataFrame ===")
print(dist_df)


=== Target Value Distribution ===
target
0     98
1     51
2    127
3    212
4     12
Name: count, dtype: int64

=== As DataFrame ===
   target  count
0       0     98
1       1     51
2       2    127
3       3    212
4       4     12
